In [1]:
from pyspark.context import SparkContext
from pyspark.conf import SparkConf

# Define your Spark application name and master URL
appName = "Test"
master = "spark://thunderobot:7077"

# Configure Spark with SparkConf
conf = SparkConf().setAppName(appName).setMaster(master)

# Create SparkContext
sc = SparkContext(conf=conf)

24/06/17 16:37:02 WARN Utils: Your hostname, thunderobot resolves to a loopback address: 127.0.1.1; using 192.168.1.201 instead (on interface wlo1)
24/06/17 16:37:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/17 16:37:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc

<SparkContext master=spark://thunderobot:7077 appName=SkillBox>

In [3]:
!curl -s "https://en.wikipedia.org/w/index.php?title=Hummingbird&action=raw" > article.txt

In [4]:
# Использовал https://github.com/apertium/WikiExtractor/blob/master/WikiExtractor.py
import requests
import re
from html.entities import name2codepoint

def clean(text):
    discardElements = set([
        'gallery', 'timeline', 'noinclude', 'pre',
        'table', 'tr', 'td', 'th', 'caption',
        'form', 'input', 'select', 'option', 'textarea',
        'ul', 'li', 'ol', 'dl', 'dt', 'dd', 'menu', 'dir',
        'ref', 'references', 'img', 'imagemap', 'source'
    ])
    
    selfClosingTags = ['br', 'hr', 'nobr', 'ref', 'references']
    ignoredTags = [
        'b', 'big', 'blockquote', 'center', 'cite', 'div', 'em',
        'font', 'h1', 'h2', 'h3', 'h4', 'hiero', 'i', 'kbd', 'nowiki',
        'p', 'plaintext', 's', 'small', 'span', 'strike', 'strong',
        'sub', 'sup', 'tt', 'u', 'var',
    ]
    placeholder_tags = {'math': 'formula', 'code': 'codice'}
    
    comment = re.compile(r'<!--.*?-->', re.DOTALL)
    discard_element_patterns = [re.compile(r'<\s*%s\b[^>]*>.*?<\s*/\s*%s>' % (tag, tag), re.DOTALL | re.IGNORECASE) for tag in discardElements]
    ignored_tag_patterns = [(re.compile(r'<\s*%s\b[^>]*>' % tag, re.IGNORECASE), re.compile(r'<\s*/\s*%s>' % tag, re.IGNORECASE)) for tag in ignoredTags]
    selfClosing_tag_patterns = [re.compile(r'<\s*%s\b[^/]*/\s*>' % tag, re.DOTALL | re.IGNORECASE) for tag in selfClosingTags]
    placeholder_tag_patterns = [(re.compile(r'<\s*%s(\s*| [^>]+?)>.*?<\s*/\s*%s\s*>' % (tag, tag), re.DOTALL | re.IGNORECASE), repl) for tag, repl in list(placeholder_tags.items())]
    preformatted = re.compile(r'^ .*?$', re.MULTILINE)
    externalLink = re.compile(r'\[\w+.*? (.*?)\]')
    externalLinkNoAnchor = re.compile(r'\[\w+[&\]]*\]')
    bold_italic = re.compile(r"'''''([^']*?)'''''")
    bold = re.compile(r"'''(.*?)'''")
    italic_quote = re.compile(r"''\"(.*?)\"''")
    italic = re.compile(r"''([^']*)''")
    quote_quote = re.compile(r'""(.*?)""')
    spaces = re.compile(r' {2,}')
    dots = re.compile(r'\.{4,}')

    def unescape(text):
        def fixup(m):
            text = m.group(0)
            code = m.group(1)
            try:
                if text[1] == "#":
                    return chr(int(code[1:], 16)) if text[2] == "x" else chr(int(code))
                else:
                    return chr(name2codepoint[code])
            except:
                return text
        return re.sub("&#?(\w+);", fixup, text)

    def dropNested(text, openDelim, closeDelim):
        openRE = re.compile(openDelim)
        closeRE = re.compile(closeDelim)
        matches = []
        nest = 0
        start = openRE.search(text, 0)
        if not start:
            return text
        end = closeRE.search(text, start.end())
        next = start
        while end:
            next = openRE.search(text, next.end())
            if not next:
                while nest:
                    nest -= 1
                    end0 = closeRE.search(text, end.end())
                    if end0:
                        end = end0
                    else:
                        break
                matches.append((start.start(), end.end()))
                break
            while end.end() < next.start():
                if nest:
                    nest -= 1
                    last = end.end()
                    end = closeRE.search(text, end.end())
                    if not end:
                        if matches:
                            span = (matches[0][0], last)
                        else:
                            span = (start.start(), last)
                        matches = [span]
                        break
                else:
                    matches.append((start.start(), end.end()))
                    start = next
                    end = closeRE.search(text, next.end())
                    break
            if next != start:
                nest += 1
        res = ''
        start = 0
        for s, e in matches:
            res += text[start:s]
            start = e
        res += text[start:]
        return res

    def dropSpans(matches, text):
        matches.sort()
        res = ''
        start = 0
        for s, e in matches:
            res += text[start:s]
            start = e
        res += text[start:]
        return res

    wikiLink = re.compile(r'\[\[([^[]*?)(?:\|([^[]*?))?\]\](\w*)')
    parametrizedLink = re.compile(r'\[\[.*?\]\]')

    acceptedNamespaces = ["", "Main", "Talk", "User", "User_talk", "Wikipedia", "Wikipedia_talk", "File", "File_talk", "MediaWiki", "MediaWiki_talk", "Template", "Template_talk", "Help", "Help_talk", "Category", "Category_talk", "Portal", "Portal_talk", "Book", "Book_talk", "Draft", "Draft_talk", "Education Program", "Education Program_talk", "TimedText", "TimedText_talk", "Module", "Module_talk", "Gadget", "Gadget_talk", "Gadget definition", "Gadget definition_talk", "Special", "Media"]

    def make_anchor_tag(match):
        link = match.group(1)
        colon = link.find(':')
        if colon > 0 and link[:colon] not in acceptedNamespaces:
            return ''
        trail = match.group(3)
        anchor = match.group(2) if match.group(2) else link
        anchor += trail
        return anchor

    text = dropNested(text, r'{{', r'}}')
    text = dropNested(text, r'{\|', r'\|}')
    text = wikiLink.sub(make_anchor_tag, text)
    text = parametrizedLink.sub('', text)
    text = externalLink.sub(r'\1', text)
    text = externalLinkNoAnchor.sub('', text)
    text = bold_italic.sub(r'\1', text)
    text = bold.sub(r'\1', text)
    text = italic_quote.sub(r'&quot;\1&quot;', text)
    text = italic.sub(r'&quot;\1&quot;', text)
    text = quote_quote.sub(r'\1', text)
    text = text.replace("'''", '').replace("''", '&quot;')
    text = unescape(text)
    text = unescape(text)
    matches = []
    for m in comment.finditer(text):
        matches.append((m.start(), m.end()))
    for pattern in selfClosing_tag_patterns:
        for m in pattern.finditer(text):
            matches.append((m.start(), m.end()))
    for left, right in ignored_tag_patterns:
        for m in left.finditer(text):
            matches.append((m.start(), m.end()))
        for m in right.finditer(text):
            matches.append((m.start(), m.end()))
    text = dropSpans(matches, text)
    for pattern in discard_element_patterns:
        text = pattern.sub('', text)
    for pattern, placeholder in placeholder_tag_patterns:
        index = 1
        for match in pattern.finditer(text):
            text = text.replace(match.group(), '%s_%d' % (placeholder, index))
            index += 1
    text = text.replace('<<', 'Â«').replace('>>', 'Â»')
    text = text.replace('canâ€™t', 'can\'t').replace('canâ€\\x99t', 'can\'t')
    text = dots.sub('...', text)
    text = spaces.sub(' ', text)
    text = preformatted.sub('', text)
    return text.strip()

def fetch_and_clean_wikipedia_article(url):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    text = response.text
    cleaned_text = clean(text)
    return cleaned_text

# Usage in Jupyter Notebook
url = "https://en.wikipedia.org/w/index.php?title=Hummingbird&action=raw"
cleaned_article = fetch_and_clean_wikipedia_article(url)
print(cleaned_article)


Hummingbirds are birds native to the Americas and comprise the biological family Trochilidae. With approximately 366 species and 113 genera, they occur from Alaska to Tierra del Fuego, but most species are found in Central and South America. As of 2024, 21 hummingbird species are listed as endangered or critically endangered, with numerous species declining in population.

Hummingbirds have varied specialized characteristics to enable rapid, maneuverable flight: exceptional metabolic capacity, adaptations to high altitude, sensitive visual and communication abilities, and long-distance migration in some species. Among all birds, male hummingbirds have the widest diversity of plumage color, particularly in blues, greens, and purples. Hummingbirds are the smallest mature birds, measuring in length. The smallest is the bee hummingbird, which weighs less than , and the largest is the giant hummingbird, weighing . Noted for long beaks, hummingbirds are specialized for feeding on flower nect

In [5]:
# Начало и конец текста
start_marker = "Hummingbirds are birds native to the Americas"
end_marker = "inspired by hummingbird wing feathers"

# Поиск по индексу
start_idx = cleaned_article.find(start_marker)
end_idx = cleaned_article.find(end_marker)

# Вырезка
text = cleaned_article[start_idx:end_idx]

# Вывод фрагмента для проверки
print(text[:500]) 

Hummingbirds are birds native to the Americas and comprise the biological family Trochilidae. With approximately 366 species and 113 genera, they occur from Alaska to Tierra del Fuego, but most species are found in Central and South America. As of 2024, 21 hummingbird species are listed as endangered or critically endangered, with numerous species declining in population.

Hummingbirds have varied specialized characteristics to enable rapid, maneuverable flight: exceptional metabolic capacity, a


In [6]:
# Parallelize очищенного текста
words = sc.parallelize(text.split())

# отображаем каждое слово на пару (слово, 1) и затем сворачиваем по ключу для подсчет и сортируем
word_counts = words.map(lambda word: (word.strip().lower(), 1)) \
                   .reduceByKey(lambda a, b: a + b) \
                   .sortBy(lambda pair: pair[1], ascending=False)

# топ-30 самых часто встречаемых
top_30_words = word_counts.take(30)

# топ-30 наименее встречаемых слов (редких)
bottom_30_words = word_counts.takeOrdered(30, lambda pair: pair[1])


In [7]:
print(top_30_words)

[('the', 497), ('of', 349), ('and', 292), ('to', 239), ('in', 231), ('a', 190), ('hummingbirds', 169), ('as', 127), ('hummingbird', 113), ('for', 107), ('are', 101), ('with', 80), ('is', 76), ('from', 75), ('their', 74), ('that', 71), ('by', 69), ('species', 64), ('have', 58), ('at', 52), ('during', 49), ('nectar', 49), ('be', 41), ('or', 40), ('on', 35), ('an', 34), ('which', 33), ('not', 31), ('when', 29), ('it', 28)]


In [8]:
print(bottom_30_words)

[('21', 1), ('noted', 1), ('rates,', 1), ('lengths)', 1), ('warm-blooded', 1), ('bladelike', 1), ('claws', 1), ('(by', 1), ('differ', 1), ('ornament', 1), ('thumb|each', 1), ('cup', 1), ('(image)', 1), ('remote', 1), ('4', 1), ('international', 1), ('nature', 1), ('declared', 1), ('vigors', 1), ('europe', 1), ('scientifically', 1), ('securely', 1), ('drawer', 1), ('clay', 1), ('assignable', 1), ('jungornithidae,', 1), ('biggest', 1), ('happened', 1), ('partly', 1), ('miocene,', 1)]


In [9]:
import csv

# Сохранение топ-30 самых часто встречаемых
with open('top_30_words.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['word', 'count'])
    for word, count in top_30_words:
        writer.writerow([word, count])

# топ-30 наименее встречаемых слов (редких)
with open('bottom_30_words.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['word', 'count'])
    for word, count in bottom_30_words:
        writer.writerow([word, count])
